# Sorption
<strong>J. Maas @ IfB, ETH Zürich, v. 2026-01-19</strong>

<div class='alert alert-info'>
    Select ▸▸ <strong>Restart the kernel and run all cells</strong> to generate the sorption isotherm and hygroexpansion coefficients.
</div>

In [ ]:
# Required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import ipywidgets as widgets
import re
from pathlib import Path

# Magic commands
%matplotlib widget

Subsequent cell imports the notebook with the data.

In [ ]:
# Import worksheets
input_folder = Path('data/05_sorption')
input_file = input_folder / 'sorption_hygroexpansion.ods'
sheets = pd.read_excel(input_file, sheet_name=None, engine='odf', header=[0,1])

# Export figures
figures_folder = Path('data/05_sorption')

The next cell determines the sheets for sorption, hygroexpansion, and density measurements. It follows manually defined patterns.

In [ ]:
# Function for filtering list of sheets
def filter_sheets(sheets, pattern):
    filtered_sheets = []
    for sheet in sheets:
        match = re.search(fr'{pattern}', sheet)
        if not match is None:
            filtered_sheets.append(match.group())
    return filtered_sheets

# Filter sheets
sheets_sorption = filter_sheets(sheets.keys(), 'Sorp_(\d+)_RH(\d+)')
sheets_hygroexpansion = filter_sheets(sheets.keys(), 'HExp_(\d+)_RH(\d+)')
sheet_density = 'Density'
sheet_climate = 'Climates'

# Re-import certain sheets for correct cell adjustment
sheets[sheet_climate] = pd.read_excel(input_file, sheet_name=sheet_climate, engine='odf').set_index('Table')

# Show sheet names
print(f'Sorption sheet names:\n{sheets_sorption}\n')
print(f'Hygroexpansion sheet names:\n{sheets_hygroexpansion}\n')
print(f"Density sheet name:\n'{sheet_density}'\n")
print(f"Climates sheet name:\n'{sheet_climate}'")

## Explore relative humidities
The true relative humidities is recorded with climate sticks. Subsequent section reads them out. You can use the widget to determine the average climate.

In [ ]:
# File paths
csv_files = list( input_folder.glob('*_raw_*.csv') )
txt_files = list( input_folder.glob('*_raw_*.txt') )

csv_files.sort()
txt_files.sort()

calibration_files = csv_files + txt_files

# Visualization function
def explore_climate(file, interval, ax):
    # Read data
    if file.suffix == '.csv':
        # Voltcraft climate sticks
        df_file = pd.read_csv(file, header=7, parse_dates=True, date_format='%d-%m-%Y %H:%M:%S', index_col='Record Time')
    elif file.suffix == '.txt':
        # EasyLog climate sticks
        df_file = pd.read_csv(file, encoding='ISO-8859-1', index_col='Time', usecols=[1, 2, 3, 4], parse_dates=True)
    else:
        raise ValueError(f'No supported file suffix "{file.suffix}" for file "{file.name}".')
    
    # Plot data
    ax.cla()
    df_file.plot(ax=ax, grid=True, ylim=[0, 100], title=f'Data: {file.name}', xlabel='Date [YYYY-MM-DD]', ylabel='Data value [?]')
    
    # Plot interval
    left_border = df_file.index[round((len(df_file)-1)*interval[0])]
    right_border = df_file.index[round((len(df_file)-1)*interval[1])]
    plt.fill_betweenx(y=[0, 100], x1=left_border, x2=right_border, ec='r', fc='#ffdddd', alpha=0.5)
    
    # Determine mean climate in interval
    df_info = df_file.loc[left_border:right_border].describe()
    display(df_info.loc[['mean','std']])
    [ax.axhline(df_info.loc['mean', c], c='k', ls=':', lw=1, zorder=3) for c in df_info.columns];

# Call widget
fig, ax_explore = plt.subplots()
widgets.interact(explore_climate, file=calibration_files, interval=widgets.FloatRangeSlider(value=[0.20, 0.95], min=0.0, max=1.0, step=0.01), ax=widgets.fixed(ax_explore));

Subsequent cell calculates the signal of one climate stick into the signal of another climate stick, based on provided calibration data from <i>data/calibration_linear_least_square.pdf</i>.

In [ ]:
# Calibration functions
funs = {
    'el-3.txt': lambda x: -31.1877 + 1.4692*x,
    'el-4.txt': lambda x: -32.2231 + 1.5799*x,
    'el-6.txt': lambda x: -31.4155 + 1.5242*x,
    'el-10.txt': lambda x: -27.7662 + 1.3962*x,
    'vc-2065li.csv': lambda x: -22.4983 + 1.3157*x,
    'vc-macro.csv': lambda x: -12.3288 + 1.2239*x,
    'vc-meso.csv': lambda x: -8.8789 + 1.1868*x
}

# Return relations
# input
input_signal = list(funs.keys())[0]
output_signal = list(funs.keys())[1]

# Transformation function
def get_transformation_coefficients(input_signal, output_signal):
    '''Determines the transformation of the input_signal into the output_signal.'''
    # Extract functions
    fun_input = funs[input_signal]
    fun_output = funs[output_signal]

    # Determine fitting points
    x = np.linspace(0, 100, 100)
    y_input = fun_input(x)
    y_output = fun_output(x)

    # Calulcate relation by least-squares fit
    fun_transform = lambda x, p: p[0] + p[1]*x
    S = lambda p: np.sum( (fun_transform(y_input, p)-y_output)**2 )
    sol = sp.optimize.minimize(S, x0=[1.0, 0.0], method='Nelder-Mead')

    # Return result
    if not sol.success:
        print('WARNING: NO SOLUTION FOUND!')
    else:
        str_output = f'Transformation from {input_signal.split(".")[0]} to {output_signal.split(".")[0]}: '
        str_output += f'{sol.x[0]:.4f}{sol.x[1]:+.4f}*x'
        print(str_output)

# Call widget
widgets.interact(get_transformation_coefficients, input_signal=list(funs.keys()), output_signal=list(funs.keys()));

Subsequently, define the tolerances of the climate stick.

In [ ]:
# Tolerances for VOLTCRAFT DL-121TH
cs_tol_rh = 0.03 # accuracy relative humidity
cs_tol_temp = 1.0 # accuracy temperature

## Sorption isotherm
This section determines the sorption isotherm from the data.

Subsequent cell extracts the relative humidity and corresponding mass from the corresponding worksheets.

In [ ]:
# Prepare sheets
sheets_sorption.sort()
df_0 = sheets['Sorp_16_RH00']
m_0 = df_0.loc(axis=1)[:, 'm [g]'].dropna().iloc[:,0].to_numpy()

# Extract for every sheet the important data
dfs_sorption = []
pd.set_option('future.no_silent_downcasting', True) # to suppress a Pandas warning
for i, sheet in enumerate(sheets_sorption):
    try:
        # Use climate from climate stick
        rH = sheets[sheet_climate].loc[sheet, 'RH mean']
    except:
        # Use climate from sheet name
        print(f'Warning. No climate data found for {sheet}. Derived climate from sheet name instead.')
        rH = re.match(fr'Sorp_(\d+)_RH(\d+)', sheet).groups()[1] # relative humidtiy as integer
    df = sheets[sheet].replace('#DIV/0!', np.nan) # respective spreadsheet
    climate = df.columns[-1][0]
    new_df = pd.DataFrame( data={ 'rH': float(rH),
                                  'm_mean': df[climate,'m [g]'].mean(),
                                  'm_std': df[climate,'m [g]'].std(),
                                  'rho_mean': df[climate,'ρ [kg/m³]'].mean(),
                                  'rho_std': df[climate,'ρ [kg/m³]'].std(),
                                  'omega_mean': ((df[climate,'m [g]'].dropna().to_numpy() - m_0) / m_0).mean(),
                                  'omega_std': ((df[climate,'m [g]'].dropna().to_numpy() - m_0) / m_0).std()
                                }, index=[i] )
    # print( ((df[climate,'m [g]'].dropna().to_numpy() - m_0) / m_0) ) # for debugging
    dfs_sorption.append(new_df)

# Concatenate extracted sheets
df_sorption = pd.concat(dfs_sorption)
display( df_sorption )

Next cell fits a function through the sorption curves. There are numerous formulations:
well summarized in <a href='https://doi.org/10.1007/978-3-642-73683-4'>Skaar (1988)</a>. tested several models and recommends the two-hydratic form of the Hailwood and Horribon model and the King (1960) theory. We fit the following equations:
<ol>
    <li>
        <a href='https://doi.org/10.1177/004051757704700213'>Dent (1977)</a> proposes the following basic model:
        \begin{align}
        \omega (\varphi) = \varphi / (A + B \varphi - C \varphi^2)
        \end{align}
        $\omega = (m-m_0) / m_0$ is the moisture content, i.e. the relation of sample mass $m$ to oven-dry mass $m_0$. $\varphi$ is the relative humidity ($0.0 \leq \varphi \leq 1.0$), and $A$, $B$, and $C$ and the fitting parameters of the function. According to <a href='https://doi.org/10.1007/978-3-642-73683-4'>Skaar (1988)</a>, this model is equivalent to the single hydrate model by <a href='https://doi.org/10.1039/tf946420b084'>Hailwood and Horribon (1946)</a>, but in another notation.
    </li>
    <li>
        The Wood Handbook (<a href='https://research.fs.usda.gov/treesearch/62200'>Ross, 2021</a>) and <a href='https://wfs.swst.org/index.php/wfs/article/view/740'>Simpson (1973)</a> recommend the two hydrate model by <a href='https://doi.org/10.1039/tf946420b084'>Hailwood and Horribon (1946)</a>. It has the following form:
        \begin{align}
        \omega = \omega_0 \left[\frac{K_d \varphi}{1 - K_d \varphi} + \frac{K_h K_d \varphi \cdot + 2 K_h K_h' K_d^2 \varphi^2}{1 + K_h K_d \varphi + K_h K_h' K_d^2 \varphi^2} \right]
        \end{align}
        According to <a href='https://doi.org/10.1007/978-3-642-73683-4'>Skaar (1988)</a>, $\omega_0$ is the moisture content developing a complete <q>monolayer coverage</q>. $K_h$ and $K_h'$ are the <q>equilibrium constants between first and second hydrates and dissolved water</q>, and $K_d$ is the <q>equilibirum constant between dissolved water and vapor</q>.
    </li>
    <li>
        <a href='https://wfs.swst.org/index.php/wfs/article/view/740'>Simpson (1973)</a> also recommends the model by <a href='https://eth.swisscovery.slsp.ch/permalink/41SLSP_ETH/lshl64/alma990017295030205503'>King (1960)</a>, which is:
        \begin{align}
        \omega = \frac{1.8}{M_p} \left[ \frac{B K_1 \varphi_0 \varphi}{1 + K_1 \varphi_0 \varphi} + \frac{D K_2 \varphi_0 \varphi}{1 - K_2 \varphi_0 \varphi} \right]
        \end{align}
        $B$, $D$, $K_1$, and $K_2$ are material parameters, and $M_p$ is the molecular weight of the polymer. $\varphi_0$ is the saturation vapor pressure.
    </li>
    <li>
        The Wood Handbook (<a href='https://research.fs.usda.gov/treesearch/62200'>Ross, 2021</a>) recommends the equation by <a href='https://doi.org/10.2737/FPL-GTR-229'>Glass et al. (2014)</a>, with the form of the equation originally from Henderson (1952):
        \begin{align}
        \omega = \left[ A T \left( 1 - \frac{T}{T_c} \right)^B \ln (1 - \varphi) \right]^{C T^D}
        \end{align}
        $T$ is the temperature and $A$, $B$, $C$, $D$, and $T_c$ are the material parameters
    </li>
</ol>

The borders between chemisorption, physiosorption, and capillary condensation highlighted in the graph are $6\,\%$ and $15\,\%$ RH, respectively, and taken from Niemz et al. (2017).

In [ ]:
# Define Least-squares fitting
def lsq_analysis(fun, exp_phi, exp_omega, p0):
    '''
    Fit function over experimental data
    Input:
        fun       = function to fit in form f(phi, p) with parameters in vector p
        exp_phi   = array with experimental relative humidities
        exp_omega = array with experimental moisture contents
        p0        = starting optimization vector p0
        ax        = pass ax for plotting result, skips plotting if None (default)
    Output:
        p         = array with determined parameters
        R2        = coefficient of determination
    '''
    # Perform least-squares fit
    S = lambda p: np.sum( ( fun(exp_phi, p) - exp_omega )**2 )
    sol = sp.optimize.minimize(S, x0=p0, method='Nelder-Mead', options={'maxiter':10000})
    if not sol.success:
        print('\033[91mWarning: Optimization did not converge.\033[00m')
    p = sol.x
    print(sol)

    # Calculate R2
    SS_res = sol['fun']
    SS_tot = np.sum( ( np.mean(exp_omega) - exp_omega )**2 )
    R2 = 1 - SS_res / SS_tot
    
    return p, R2


def fit_isotherm(fun, exp_phi, exp_omega, p0, ax=None, name='Function', line_format=dict()):
    # Separate adsorption / desorption
    phi_adsorp, phi_desorp = exp_phi[0], exp_phi[1]
    omega_adsorp, omega_desorp = exp_omega[0], exp_omega[1]
    
    # Determine adsorption
    print(name.upper() + ':')
    sol_adsorp, R2_adsorp = lsq_analysis( fun, phi_adsorp, omega_adsorp, p0 )
    print(f'Adsorption parameters: {sol_adsorp}')

    # Determine desorption parameters
    phi_desorp = np.insert(phi_desorp.to_numpy(), 0, 1.0) # include RH = 1.0
    omega_desorp = np.insert(omega_desorp.to_numpy(), 0, fun(1.0, sol_adsorp)) # include calculated endpoint of adsorption curve
    sol_desorp, R2_desorp = lsq_analysis( fun, phi_desorp, omega_desorp, p0 )
    print(f'Desorption parameters: {sol_desorp}')

    # Add result to plot
    if not ax is None:
        x = np.linspace(0.0, 1.0, 100)
        y_adsorp = fun(x, sol_adsorp)
        y_desorp = fun(x, sol_desorp)
        
        ax.plot(x, y_adsorp, ls='-', label=f'{name} (Adsorption)', **line_format)
        ax.plot(x, y_desorp, ls='--', label=f'{name} (Desorption)', **line_format)
        
    print(f'M.C. fiber saturation: {np.mean([fun(1.0, sol_adsorp), fun(1.0, sol_desorp)]):.6f}')
    return sol_adsorp, sol_desorp, R2_adsorp, R2_desorp


def draw_reference(ax, ref):
    # Sorption isotherm from Wood Handbook (Ross, 2021) for Sitka spruce, obtained from oscillating vapor pressure desorption
    fun_ross = lambda phi, p: 18./p[0] * ( (p[1]*phi)/(1-p[1]*phi) + (p[2]*p[1]*phi+2*p[3]*p[2]*p[1]**2*phi**2)/(1+p[2]*p[1]*phi+p[3]*p[2]*p[1]**2*phi**2) )
    T = 23. # temperature in [°C]
    W = 349.+1.29*T+0.0135*T**2
    K = 0.805+0.000736*T-0.00000273*T**2
    K1 = 6.27-0.00938*T-0.000303*T**2
    K2 = 1.91+0.0407*T-0.000293*T**2
    p_ross = [W, K, K1, K2]

    # Sorption isotherm from Hansen (1986) for spruce
    fun_hansen = lambda phi, p: p[0] * np.exp( (-1/p[1]) * np.log(1-np.log(phi)/p[2]) ) / 100.
    p_hansen_adsorp = [3.37e1, 1.95, 6.26e-2]
    p_hansen_desorp = [3.09e1, 1.05, 3.83e-1]

    # Sorption isotherm from Skaar (1988) for ten North American wood species
    fun_skaar = lambda phi, p: phi / ( p[0] + p[1]*phi - p[2]*phi**2 )
    p_skaar_adsorp = [1.64, 14.3, 12.1]
    p_skaar_desorp = [2.23, 8.3, 7.0]

    # Select reference
    match ref:
        case 'wood handbook':
            fun = fun_ross
            p0 = (p_ross, 'Reference (Ross, 2021)')
            p1 = None
        case 'hansen':
            fun = fun_hansen
            p0 = (p_hansen_adsorp, 'Hansen (1986) (Adsorption)')
            p1 = (p_hansen_desorp, 'Hansen (1986) (Desorption)')
        case 'skaar':
            fun = fun_skaar
            p0 = (p_skaar_adsorp, 'Skaar (1988) (Adsorption)')
            p1 = (p_skaar_desorp, 'Skaar (1988) (Desorption)')
        case _:
            raise ValueError(f'Unknown reference {ref}')
            
    # Draw line
    x = np.linspace(0, 1, 100)
    if not p0 is None: ax.plot(x, fun(x, p0[0]), label=p0[1], ls='-', c='lightgray', zorder=1 )
    if not p1 is None: ax.plot(x, fun(x, p1[0]), label=p1[1], ls='--', c='lightgray', zorder=1 )

    return ax


# Select data of adsorption and desorption path manually
idx_adsorp = [17] + list(range(2, 9))
idx_desorp = np.arange(9, 16)

rh_adsorp = df_sorption.iloc[idx_adsorp]['rH']/100.
omega_adsorp = df_sorption.iloc[idx_adsorp]['omega_mean']
omega_std_adsorp = df_sorption.iloc[idx_adsorp]['omega_std']

rh_desorp = df_sorption.iloc[idx_desorp]['rH']/100.
omega_desorp = df_sorption.iloc[idx_desorp]['omega_mean']
omega_std_desorp = df_sorption.iloc[idx_desorp]['omega_std']

# Define plot function
def plot_isotherm(ax):
    # Initialize plot
    ax.grid()
    # ax.set_ylim([-0.015, 0.295])
    # ax.set_title('Sorption isotherm')
    ax.set_xlabel(r'Relative humidity $\varphi\ [-]$', fontsize='large')
    ax.set_ylabel(r'Moisture content $\omega\ [-]$', fontsize='large')
    
    # Plot experimental data
    # ax.plot(rh_adsorp, omega_adsorp, 'ko', label='Data points (Adsorption)', fillstyle='none')
    # ax.plot(rh_desorp, omega_desorp, 'k^', label='Data points (Desorption)', fillstyle='none')
    ax.errorbar(rh_adsorp, omega_adsorp, yerr=omega_std_adsorp, xerr=cs_tol_rh, label='Data (Adsorption)', marker='o', c='k', ls='', fillstyle='none')
    ax.errorbar(rh_desorp, omega_desorp, yerr=omega_std_desorp, xerr=cs_tol_rh, label='Data (Desorption)', marker='s', c='k', ls='', fillstyle='none')
    
    # Fit Hailwood-Horribon function
    fun_dent = lambda phi, p: phi / (p[0] + p[1]*phi - p[2]*phi**2) # p = [A, B, C]
    fun_HH = lambda phi, p: p[0] * ( (p[1]*phi) / (1-p[1]*phi)  +  (p[2]*p[1]*phi+2*p[2]*p[3]*p[1]**2*phi**2) / (1+p[2]*p[0]*phi+p[2]*p[3]*p[1]**2*phi**2) ) # p = [omega_0, K_d, K_h, K_h']
    fun_king = lambda phi, p: 1.800/p[0] * ( (p[1]*p[3]*phi*p[5])/(1+p[3]*phi*p[5]) + (p[2]*p[4]*phi*p[5])/(1-p[4]*phi*p[5]) ) # p = [M_p, B, D, K_1, K_2, phi_0]
    fun_glass, T = lambda phi, p: ( p[0] * T * (1-T/p[4])**p[1] * np.log(1-phi) )**(p[2]*T**p[3]), 293.15 # p = [A, B, C, D, T_c]
    
    sol_dent = fit_isotherm(fun_dent, (rh_adsorp, rh_desorp), (omega_adsorp, omega_desorp), p0=[1.6, 14.3, 12.1], ax=ax, name='One-hydrate HH model', line_format={'c': 'k'})
    # sol_HH = fit_isotherm(fun_HH, (rh_adsorp, rh_desorp), (omega_adsorp, omega_desorp), p0=[0.005, 0.805, 6.27, 1.91], ax=ax, name='Two-hydrate HH model', line_format={'c': 'gray'})
    # sol_king = fit_isotherm(fun_king, (rh_adsorp, rh_desorp), (omega_adsorp, omega_desorp), p0=[255, 1.58, 0.612, 6.91e5, 2.39e5, 0.35e-5], ax=ax, name='King', line_format={'c': 'lightgray'})
    # sol_glass = fit_isotherm(fun_glass, (rh_adsorp, rh_desorp), (omega_adsorp, omega_desorp), p0=[-6.12e-4, 2.43, 0.0577, 0.43, 647.1], ax=ax, name='Glass', line_format={'c': '#1111dd', 'lw': 0.8})
    # draw_reference(ax, 'wood handbook')
    
    # Add annotations
    annot_adsorp = fr'Adsorption: ($R^2 = {sol_dent[2]:.2f}$)' + '\n' + rf'$\omega = \varphi\,/\,({sol_dent[0][0]:.3f} + {sol_dent[0][1]:.3f} \varphi - {sol_dent[0][2]:.3f} \varphi^2)$'
    ax.annotate(annot_adsorp, xy=(0.56, 0.27), xytext=(0.31, 0.14), xycoords='axes fraction', arrowprops=dict(facecolor='black', shrink=0.00, width=1, headwidth=6, headlength=6), transform=ax.transAxes, fontsize=10, va='top')
    annot_desorp = f'Desorption: ($R^2 = {sol_dent[3]:.2f}$)' + '\n' + rf'$\omega = \varphi\,/\,({sol_dent[1][0]:.3f} + {sol_dent[1][1]:.3f} \varphi - {sol_dent[1][2]:.3f} \varphi^2)$'
    ax.annotate(annot_desorp, xy=(0.75, 0.52), xytext=(0.69, 0.61), xycoords='axes fraction', arrowprops=dict(facecolor='black', shrink=0.00, width=1, headwidth=6, headlength=6), transform=ax.transAxes, fontsize=10, ha='right', ma='left')
    
    # Add sorption phases
    sorp_bnds = [0.06, 0.15] # moisture content
    sorp_txt_x = 1.02
    ax.fill_between([0, 1], [sorp_bnds[0], sorp_bnds[0]], [0, 0], color='#cccccc')
    ax.fill_between([0, 1], [sorp_bnds[0], sorp_bnds[0]], [sorp_bnds[1], sorp_bnds[1]], color='#dddddd')
    ax.fill_between([0, 1], [sorp_bnds[1], sorp_bnds[1]], [0.3, 0.3], color='#eeeeee')
    ax.text(x=sorp_txt_x, y=sorp_bnds[0]/2, s='mono-\nmolecular', ha='center', va='top', rotation=90, rotation_mode='anchor')
    ax.text(x=sorp_txt_x, y=(sorp_bnds[0]+sorp_bnds[1])/2, s='multimolecular', ha='left', va='center', rotation=90)
    ax.text(x=sorp_txt_x, y=(sorp_bnds[1]+0.3)/2, s='capillary condensation', ha='left', va='center', rotation=90)
    
    # Format axes
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 0.3])
    
    ax.legend()
    return ax


# Create figure
fig, ax = plt.subplots(figsize=(5.2, 4.5))
plot_isotherm(ax)
fig.tight_layout()
fig.savefig(figures_folder/'sorption_isotherm.pdf', bbox_inches='tight');

## Hygroexpansion
The subsequent section investigates the relationship between strain and moisture content.

For determining the swelling and shrinkage strains, subsequent cell extracts the lengths and masses of the swelling/shrinkage samples.

In [ ]:
# Extract for every sheet the important data
dfs_hygroexpansion = []
for i, sheet in enumerate(sheets_hygroexpansion):
    df = sheets[sheet]
    try:
        # Use climate from climate stick
        rH = sheets[sheet_climate].loc[sheet.replace('HExp', 'Sorp'), 'RH mean']
    except:
        # Use climate from sheet name
        print(f'Warning. No climate data found for {sheet}. Derived climate from sheet name instead.')
        rH = re.match(fr'HExp_(\d+)_RH(\d+)', sheet).groups()[1]
    climate = df.columns[-1][0]
    directions = df[df.columns[0][0], 'Direction i']
    for direction in directions.dropna().unique():
        sub_df = df.loc[directions == direction]
        new_df = pd.DataFrame( data={ 'rH': float(rH),
                                      'direction': direction,
                                      'l_mean': sub_df[climate,'li,mean [mm]'].mean(),
                                      'l_std': sub_df[climate,'li,mean [mm]'].std(),
                                      'm_mean': sub_df[climate,'m [g]'].mean(),
                                      'm_std': sub_df[climate,'m [g]'].std()
                                    }, index=[i] )
        dfs_hygroexpansion.append(new_df)

# Concatenate extracted sheets
df_hygroexpansion = pd.concat(dfs_hygroexpansion).set_index('direction', append=True)
display( df_hygroexpansion )

Next cell determines the relationship between moisture content (from the sorption isotherm) and the length of the sample. To investigate a potential hysteresis, adsorption and desorption are investigated separately. We apply the linear fit
\begin{align}
l = p_0 + p_1 \cdot \omega .
\end{align}
$l$ is the sample length, $p_0 = l_0$ is the length at $\omega = 0$, and $p_1$ is the slope of the fit.

In [ ]:
# Select adsorption / desorption rows manually
idx_adsorp = np.arange(1, 9)
idx_desorp = np.arange(9, 16)

# Determine and plot length / m.c. relationship
def fit_hygroexpansion(df, idx, axs=None, lines=None, name='', mode='length', normalize=None, exp_options=dict(), fit_options=dict(),
                       models={'R': 'linear', 'T': 'linear', 'L': 'bilinear'}):
    # Initialize
    df = df.loc[idx] # crop dataframe
    params = dict()

    # Restructure formatting options
    def restruct_kwargs(kwargs):
        format_dicts = []
        for i in range(3):
            format_dict = dict()
            for key in kwargs.keys():
                format_dict[key] = kwargs[key][i] if isinstance(kwargs[key], (list, tuple)) else kwargs[key]
            format_dicts.append(format_dict)
        return format_dicts
    exp_options = restruct_kwargs(exp_options)
    fit_options = restruct_kwargs(fit_options)

    # Iterate over all directions
    directions = df.index.get_level_values('direction').unique()
    for i, direction in enumerate(directions):
        print(f'{direction} direction:')
        
        # Extract l_mean and m.c.
        df_dir = df.xs(direction, level='direction') # extract all directions of one type
        df_dir = pd.merge(df_dir, df_sorption, on='rH') # merge columns with m.c. to sub-dataframe
        omega, omega_std, l_mean, l_std = df_dir['omega_mean'], df_dir['omega_std'], df_dir['l_mean'], df_dir['l_std']

        # Setup model
        model = models[direction]
        match model:
            case 'linear':
                fun_fitting = lambda x, p: p[0] + p[1]*x
                p0 = [l_mean.mean(), l_mean.diff().mean()]
                bounds = [(-np.inf, np.inf), (-np.inf, np.inf)]
            case 'bilinear':
                fun_fitting = lambda x, p: p[0] + (p[1]+p[2])/2*x + (p[2]-p[1])/2*np.abs(x-p[3])
                p0 = [l_mean.mean(), 10.0, 0.1, omega.mean()]
                bounds = [(0, np.inf), (0, np.inf), (0, np.inf), (0, 0.3)]
            case _:
                raise ValueError(f"Invalid model type '{model}'.")

        # Least-squares fitting
        S = lambda p: np.sum( (fun_fitting(omega, p) - l_mean)**2 )
        sol = sp.optimize.minimize(S, x0=p0, bounds=bounds, method='Nelder-Mead')
        if not sol.success:
            print('\033[91mWarning: Optimization did not converge.\033[00m')
        print(f'Parameters: {sol.x}')

        # Calculate R2
        SS_res = sol['fun']
        SS_tot = np.sum( ( np.mean(l_mean) - l_mean )**2 )
        R2 = 1 - SS_res / SS_tot
        
        # Save results
        params[direction] = {'model': model, 'p': sol.x, 'R2': R2, 'fun': fun_fitting, 'omega': omega, 'omega_std': omega_std, 'l_mean': l_mean, 'l_std': l_std}

        # Plot result
        if not axs is None:
            ax = axs[i] if isinstance(axs, (list, tuple)) else axs
            x = np.linspace(omega.min(), omega.max(), 100)
            y = fun_fitting(x, sol.x)

            if mode == 'length':
                # (ymin, ymax) = (fun_linear(0, sol.x), fun_linear(0.25, sol.x)) if normalize is None else (normalize[i][0], normalize[i][1])
                l_exp = ax.errorbar(x=omega, y=l_mean, xerr=omega_std, yerr=l_std, label=f'{name} {direction} values', fillstyle='none', **exp_options[i])
                l_fit,  = ax.plot(x, y, label=f'{name} {direction} fit', **fit_options[i])
                if not lines is None:
                    lines.append(l_exp)
                    lines.append(l_fit)
            elif mode == 'strain':
                pass
            else:
                raise ValueError(f'Unknown mode {mode}.')

    return params

# Initialize figure
fig, ax = plt.subplots(figsize=(7, 5))
ax_r = ax.twinx()
ax.set_xlim([0.0, 0.25])
ax.set_ylim([102, 110])
ax_r.set_ylim([489.4, 491.0])
ax.grid()
ax.set_title('Lengths of hygroexpansion samples')
ax.set_xlabel(r'Moisture content $\omega$ [-]')
ax.set_ylabel(r'Average length $l$ [mm] (T / R)')
ax_r.set_ylabel(r'Average length $l$ [mm] (L)')

# Call function
lines = [] # list for saving line handles
print('Adsorption:')
params_adsorp = fit_hygroexpansion(df_hygroexpansion, idx_adsorp, axs=[ax, ax, ax_r], lines=lines, name='Adsorp.',
                                   exp_options={'c':'k', 'marker': ['o', '^', 's'], 'ls': ''},
                                   fit_options={'c':'k', 'ls': ['-', '--', ':']})
print('\nDesorption:')
params_desorp = fit_hygroexpansion(df_hygroexpansion, idx_desorp, axs=[ax, ax, ax_r], lines=lines, name='Desorp.',
                                   exp_options={'c':'gray', 'marker': ['o', '^', 's'], 'ls': ''},
                                   fit_options={'c':'gray', 'ls': ['-', '--', ':']})

# Add legend
lines = lines[::2] + lines[1::2]
ax_r.legend(lines, [l.get_label() for l in lines], loc='upper left', ncols=2, fontsize=8)

fig.tight_layout()
fig.savefig(figures_folder/'hygroexpansion_lengths.pdf', bbox_inches='tight');

The next cell determines the sorption coefficients and plots the normalized strains into a diagram. We calculate it as following, with assuming the hygroexpansive strain as $\epsilon^{\omega}_i = \alpha_i \left( \min(\omega, \omega_{FS}) - \omega_0 \right)$  from <a href='https://doi.org/10.1016/j.cma.2014.10.031'>Hassani et al. (2015)</a>:
\begin{align}
l &= p_0 + p_1 \cdot \omega = l_0 + \Delta l \\
&\Rightarrow \Delta l = p_1 \cdot \Delta \omega \\
\epsilon^{\omega} &= \frac{\Delta l}{l_0} = \frac{p_1}{p_0} \cdot \Delta \omega = \alpha \cdot \Delta \omega \\
&\Rightarrow \alpha = \frac{p_1}{p_0}
\end{align}
$l$ is the sample length, $l_0$ is the length at $\omega = 0$, $\Delta l$ is the length change in relation to $l_0$, $\alpha$ is the moisture content, $\Delta \omega$ is the moisture content change, $\alpha_i$ is the hygroexpansion coefficient in direction $i = [R, T, L]$, $\omega_{FS}$ is the moisture content at fiber saturation point, and $\omega_{0}$ is the reference moisture content in a calculation. $p_0$ and $p_1$ are the fitting parameters obtained from the length.

As a comparison, the table below contains the hygroexpansion coefficients summarized by <a href='https://doi.org/10.1016/j.cma.2014.10.031'>Hassani et al. (2015)</a>.

| $\alpha_R [1/\%]$ | $\alpha_T [1/\%]$ | $\alpha_L [1/\%]$ |
| ----------------- | ----------------- | ----------------- |
| 0.00170           | 0.00330           | 0.00005           |

In [ ]:
# Determine strain parameters
params_full = fit_hygroexpansion(df_hygroexpansion, np.arange(1, 16))

# Define plotting function
def plot_hygroexpansion(params_full, ax):
    # Initialize plot
    ax_r = ax.twinx()
    ax.grid()
    
    # Axes labels
    ax.set_xlim([0.0, 0.25])
    ax.set_ylim([-0.0, 0.09])
    ax_r.set_ylim([0.0e-4, 22.5e-4])
    ax_r.ticklabel_format(axis='y', scilimits=[-3, 3])
    
    ax.set_xlabel(r'Moisture content $\omega\ [-]$', fontsize='large')
    ax.set_ylabel(r'Strain $\epsilon_{i}^{\omega}(\omega) = \dfrac{l_i (\omega) - l_i(0)}{l_i (0)}\ \text{ for }\ i\,\in\,\{ T, R \}\ [-]$', fontsize='large')
    ax_r.set_ylabel(r'Strain $\epsilon_{i}^{\omega}(\omega)\ \text{ for }\ i = L\ [-]$', fontsize='large')
    
    # Line formattings
    axs = [ax, ax, ax_r]
    markers = ['o', '^', 's']
    ls = ['-', '--', ':']
    
    # Plot lines
    print('\n Hygroexpansion coefficients [1/%]:')
    lines = []
    for i, key in enumerate( params_full.keys() ):
        # Extract parameters
        param = params_full[key]
        model, p, R2, fun = param['model'], param['p'], param['R2'], param['fun']
        omega, omega_std, l_mean, l_std = param['omega'], param['omega_std'], param['l_mean'], param['l_std']

        # Show results
        text_x, text_y = 0.98, 0.25
        match model:
            case 'linear':
                l0 = p[0]
                alpha = p[1] / l0
                print(f'    {key}: {alpha/100:.6f}')
                ax.text(text_x, text_y-i*0.06, fr'$\alpha_{key} = {alpha:.4f}$ ($R^2 = {R2:.2f}$)', transform=ax.transAxes, fontsize=10, va='top', ha='right')
            case 'bilinear':
                l0 = fun(0, p)
                alpha = p[1] / l0
                alpha_2 = p[2] / l0
                ax.text(text_x, text_y-i*0.06, fr'$\alpha_{key} (\omega \leq {p[3]:.3f}) = {alpha:.4f}$ ($R^2 = {R2:.2f}$)', transform=ax.transAxes, fontsize=10, va='top', ha='right')
                ax.text(text_x, text_y-(i+1)*0.06, fr'$\alpha_{key} (\omega > {p[3]:.3f}) = {alpha_2:.4f}$ ($R^2 = {R2:.2f}$)', transform=ax.transAxes, fontsize=10, va='top', ha='right')
            case _:
                raise ValueError(f"Invalid model '{model}'.")
    
        # Plot strains
        ax_i = axs[i]
        l_err = ax_i.errorbar(x=omega, y=(l_mean-l0)/l0, xerr=omega_std, yerr=l_std/l0, label=f'Data {key}', marker=markers[i], fillstyle='none', ls='', c='k')
        lines.append(l_err)
        
        x = np.linspace(omega.min(), omega.max(), 100)
        l_fit,  = ax_i.plot(x, (fun(x, p)-l0)/l0, label=f'Fit {key}', ls=ls[i], c='k')
        lines.append(l_fit)
    
    # Finish figure
    lines = lines[::2] + lines[1::2]
    ax_r.legend(lines, [l.get_label() for l in lines], loc='upper left', ncols=2)
    return ax
    

# Create figure
fig, ax = plt.subplots(figsize=(5.6, 4.5))
plot_hygroexpansion(params_full, ax)
fig.tight_layout()
fig.savefig(figures_folder/'hygroexpansion.pdf', bbox_inches='tight');

## Density
The subsequent section reads out the oven-dry density and the dependency of the density on the moisture content.

In [ ]:
# Extract density
df_density = sheets[sheet_density].set_index(('Climate (RH/T)', 'Sample')).dropna(axis='rows', subset=[('0%/20°C', 'ρ [kg/m³]')])['0%/20°C']
print(f'Oven-dry density:\n    Mean: {df_density.loc["Mean", "ρ [kg/m³]"]}\n    Std: {df_density.loc["Std", "ρ [kg/m³]"]}')

The next cell determines the relationship between density and moisture content.

In [ ]:
# Extract data
idx_density = np.arange(1,17)
omega, omega_std = df_sorption.iloc[idx_density]['omega_mean'], df_sorption.iloc[idx_density]['omega_std']
rho, rho_std = df_sorption.iloc[idx_density]['rho_mean'], df_sorption.iloc[idx_density]['rho_std']

# Fit linear function
fun_linear = lambda x, p: p[0] + p[1]*x
S = lambda p: np.sum( (fun_linear(omega, p) - rho)**2 )
sol = sp.optimize.minimize(S, x0=[rho.mean(), rho.diff().mean()/omega.diff().mean()], method='Nelder-Mead')
if not sol.success:
    print('\033[91mWarning: Optimization did not converge.\033[00m')

# Normalize results
rho /= sol.x[0]
rho_std /= sol.x[0]
sol.x /= sol.x[0]

# Plot density over moisture content
fig, ax = plt.subplots(figsize=(6,4.5))
ax.errorbar(x=omega, y=rho, xerr=omega_std, yerr=rho_std, label='Data', marker='o', c='k', ls='', fillstyle='none' )

x = np.linspace(omega.min(), omega.max(), 100)
ax.plot(x, fun_linear(x, sol.x), label='Fit', color='k', ls='--')

# Annotations
ax.set_xlabel(r'Moisture content $\omega\ [-]$')
ax.set_ylabel(r'Normalized density $\rho\,(\omega)\,/\,\rho_0\ [-]$')
ax.set_xlim([-0.01, 0.24])
ax.set_ylim([0.94, 1.17])
# ax.set_ylim([320, 400]) # for non-normalized results

eq_string = fr'$\rho(\omega)\,/\,\rho_0 =  {sol.x[0]:.1f} + {sol.x[1]:.6f}\,\omega$'
print(eq_string)
ax.text(0.94, 0.16, eq_string, transform=ax.transAxes, fontsize=10, verticalalignment='bottom', horizontalalignment='right')

# Format plot
ax.legend(loc='upper left')
ax.grid(which='major')
ax.grid(which='minor', color='gray', linestyle='-', alpha=0.2)
plt.minorticks_on()

fig.tight_layout()
fig.savefig(figures_folder/'density.pdf', bbox_inches='tight');

## Summary
The next cell plots the data together.

In [ ]:
# Initialize figure
fig, axs = plt.subplots(1, 2, figsize=(10.6, 4.5))

# Draw diagrams
plot_isotherm(axs[0])
plot_hygroexpansion(params_full, axs[1])

# Finish up figure
fig.tight_layout()
fig.savefig(figures_folder/'sorption_results.png', bbox_inches='tight', dpi=300);

## References
<ul>
    <li><a href='https://doi.org/10.1177/004051757704700213'>Dent, R.W., 1977. A Multilayer Theory for Gas Sorption: Part I: Sorption of a Single Gas. Text. Res. J. 47, 145–152.</a></li>
    <li><a href='https://doi.org/10.2737/FPL-GTR-229'>Glass, S.V., Zelinka, S.L., Johnson, J.A., 2014. Investigation of Historic Equilibrium Moisture Content Data from the Forest Products Laboratory. USDA Forest Service, Forest Products Laboratory, General Technical Report, FPL-GTR-229, 2014; 37 p. 229, 1–37.</a></li>
    <li><a href='https://doi.org/10.1039/tf946420b084'>Hailwood, A.J., Horrobin, S., 1946. Absorption of water by polymers: analysis in terms of a simple model. Trans. Faraday Soc. 42, 84–92.</a></li>
    <li><a href='https://orbit.dtu.dk/en/publications/sorption-isotherms-a-catalogue'>Hansen, K.K., 1986. Sorption isotherms: A catalogue (Byg Rapport). Technical University of Denmark.</a></li>
    <li><a href='https://doi.org/10.1016/j.cma.2014.10.031'>Hassani, M.M., Wittel, F.K., Hering, S., Herrmann, H.J., 2015. Rheological model for wood. Comput. Methods Appl. Mech. Eng. 283, 1032–1060.</a></li>
    <li>Henderson, S.M. 1952. A basic concept of equilibrium moisture. Agricultural Engineering. 33:29–32.</li>
    <li><a href='https://eth.swisscovery.slsp.ch/permalink/41SLSP_ETH/lshl64/alma990017295030205503'>King, G., 1960. Theories of multi-layer adsorption, in: Hearle, J.W.S., Peters, R.H. (Eds.), Moisture in Textiles. The Textile Institute, Manchester, p. IX, 203.</a></li>
    <li><a href='https://www.hanser-elibrary.com/doi/book/10.3139/9783446445468'>Niemz, P., Sonderegger, W., 2017. Holzphysik: Physik des Holzes und der Holzwerkstoffe. Carl Hanser Verlag GmbH & Co. KG, Munich, Germany.</a></li>
    <li><a href='https://research.fs.usda.gov/treesearch/62200'>Ross, R. (Ed.), 2021. Wood handbook - Wood as an engineering material, General Technical Report FPL-GTR-282. ed. U.S. Department of Agriculture, Forest Service, Forest Products Laboratory, Madison, WI.</a></li>
    <li><a href='https://wfs.swst.org/index.php/wfs/article/view/740'>Simpson, W.T., 1973. Predicting Equilibrium Moisture Content of Wood by Mathematical Models. Wood and Fiber Science 41–49.</a></li>
    <li><a href='https://doi.org/10.1007/978-3-642-73683-4'>Skaar, C., 1988. Wood-water relations, Springer series in wood science. Springer, Berlin u.a.</a></li>
</ul>